# Phase 0: Qualitative Verification of Introspection in Gemma 3 IT

Before building full SAE feature analysis infrastructure, we need to verify that Gemma 3 shows introspection signal similar to what we observed in Qwen models.

**Goal:** Run injection + control trials and manually assess whether the model can detect injected concepts.

**Go/no-go criteria:**
- If clear signal (model reports detecting something, names concept): proceed to Phase 1
- If weak/no signal: try larger model before abandoning
- If high false positives: adjust prompt or injection strength

**Model options:**
- Local testing: `google/gemma-3-270m-it` (layers 5, 9, 12, 15)
- Colab/A100: `google/gemma-3-4b-it` (layers 7, 13, 17, 22, 27, 32)

## Setup

In [1]:
!nvidia-smi

Sun Feb  8 04:05:33 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             50W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
%pip install sae-lens transformer-lens python-dotenv numpy pandas --upgrade

  Using cached numpy-2.4.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)


In [ ]:
from dotenv import load_dotenv
import os

# Load HF token from Google Drive .env file
load_dotenv("drive/MyDrive/open_introspection/.env")

from huggingface_hub import login
login(token=os.environ["HF_TOKEN"])

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
import pandas as pd
from IPython.display import display, HTML, Markdown
from tqdm.auto import tqdm
from transformers import AutoTokenizer

torch.set_grad_enabled(False)

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

Device: cuda


## Model Configuration

Choose model size based on available compute:
- **270m**: Fast local testing (~500MB)
- **1b**: Local with decent GPU (~2GB)
- **4b**: Colab/A100 recommended (~8GB)

**Critical:** Must use bfloat16 - float16 causes NaN issues with Gemma 3.

In [24]:
# === CHOOSE MODEL SIZE ===
MODEL_SIZE = "4b"  # Options: "270m", "1b", "4b", "12b"

# Model and SAE configuration
MODEL_CONFIGS = {
    "270m": {
        "model": "google/gemma-3-270m-it",
        "sae_release": "gemma-scope-2-270m-it-res",
        "layers": [5, 9, 12, 15],
        "injection_layer": 12,  # ~2/3 through
        "n_layers": 18,
    },
    "1b": {
        "model": "google/gemma-3-1b-it",
        "sae_release": "gemma-scope-2-1b-it-res",
        "layers": [7, 13, 17, 22],
        "injection_layer": 17,  # ~2/3 through (26 layers)
        "n_layers": 26,
    },
    "4b": {
        "model": "google/gemma-3-4b-it",
        "sae_release": "gemma-scope-2-4b-it-res",
        "layers": [9, 17, 22, 29],  # Corrected: actual available layers
        "injection_layer": 20,  # ~2/3 through (34 layers)
        "n_layers": 34,
    },
    "12b": {
        "model": "google/gemma-3-12b-it",
        "sae_release": "gemma-scope-2-12b-it-res",
        "layers": [10, 20, 30, 40],  # Check actual available layers
        "injection_layer": 30,
        "n_layers": 48,
    },
}

config = MODEL_CONFIGS[MODEL_SIZE]
MODEL_NAME = config["model"]
SAE_RELEASE = config["sae_release"]
INJECTION_LAYER = config["injection_layer"]

print(f"Model: {MODEL_NAME}")
print(f"SAE release: {SAE_RELEASE}")
print(f"Injection layer: {INJECTION_LAYER}")
print(f"Available SAE layers: {config['layers']}")

Model: google/gemma-3-4b-it
SAE release: gemma-scope-2-4b-it-res
Injection layer: 20
Available SAE layers: [9, 17, 22, 29]


In [7]:
from sae_lens import HookedSAETransformer

model = HookedSAETransformer.from_pretrained(
    MODEL_NAME,
    device=device,
    dtype=torch.bfloat16  # Critical: Gemma 3 requires bfloat16
)

# Load tokenizer separately for chat template
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"Model loaded: {model.cfg.n_layers} layers, d_model={model.cfg.d_model}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Loaded pretrained model google/gemma-3-4b-it into HookedTransformer
Model loaded: 34 layers, d_model=2560


## Chat Template Helper

Gemma 3 IT uses a specific chat format. The system message gets combined with the user message.

In [8]:
def format_chat(system: str, user: str) -> str:
    """Format messages using Gemma chat template."""
    if system:
        content = f"SYSTEM INSTRUCTIONS:{system}\n\nUSER INPUT:{user}"
    else:
        content = user
    messages = [{"role": "user", "content": content}]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

# Get the end_of_turn token ID for stopping generation
END_OF_TURN_ID = tokenizer.encode("<end_of_turn>", add_special_tokens=False)[0]
print(f"End of turn token ID: {END_OF_TURN_ID}")

# Test
test_prompt = format_chat("You are helpful.", "Say hi.")
print(f"\nWith system prompt:\n{test_prompt}")

test_no_sys = format_chat("", "Tell me about silence")
print(f"\nWithout system prompt:\n{test_no_sys}")

End of turn token ID: 106

With system prompt:
<bos><start_of_turn>user
SYSTEM INSTRUCTIONS:You are helpful.

USER INPUT:Say hi.<end_of_turn>
<start_of_turn>model


Without system prompt:
<bos><start_of_turn>user
Tell me about silence<end_of_turn>
<start_of_turn>model



## Concept Extraction

Extract the "silence" concept vector using mean subtraction:
1. Run "Tell me about silence" through model
2. Run baseline words through model
3. Subtract mean baseline activations from concept activations

This gives us a direction in activation space representing the concept.

In [9]:
# Configuration
CONCEPT = "silence"
HOOK_NAME = f"blocks.{INJECTION_LAYER}.hook_resid_post"


# TODO: Get the larger set of words used in existing study.
# Baseline words for mean subtraction (neutral, common words)
BASELINE_PROMPTS = [
    "Tell me about water",
    "Tell me about tables",
    "Tell me about walking",
    "Tell me about numbers",
    "Tell me about buildings",
]

CONCEPT_PROMPT = f"Tell me about {CONCEPT}"

print(f"Concept: {CONCEPT}")
print(f"Injection layer: {INJECTION_LAYER} (hook: {HOOK_NAME})")

Concept: silence
Injection layer: 22 (hook: blocks.22.hook_resid_post)


In [10]:
def extract_activations(model, prompt: str, layer: int) -> torch.Tensor:
    """Extract residual stream activations at a specific layer."""
    # Format as chat — no system message for concept extraction
    chat_prompt = format_chat("", prompt)
    # prepend_bos=False because apply_chat_template already includes <bos>
    tokens = model.to_tokens(chat_prompt, prepend_bos=False)
    _, cache = model.run_with_cache(tokens, names_filter=[f"blocks.{layer}.hook_resid_post"])
    # Return activations at the last token position
    return cache[f"blocks.{layer}.hook_resid_post"][0, -1, :]


def extract_concept_vector(model, concept_prompt: str, baseline_prompts: list[str], layer: int) -> torch.Tensor:
    """Extract concept vector using mean subtraction."""
    # Get concept activations
    concept_acts = extract_activations(model, concept_prompt, layer)
    
    # Get baseline activations
    baseline_acts = torch.stack([
        extract_activations(model, prompt, layer)
        for prompt in tqdm(baseline_prompts, desc="Extracting baselines")
    ])
    
    # Mean subtraction
    baseline_mean = baseline_acts.mean(dim=0)
    concept_vector = concept_acts - baseline_mean
    
    return concept_vector

In [11]:
# Extract the silence concept vector
concept_vector = extract_concept_vector(
    model, 
    CONCEPT_PROMPT, 
    BASELINE_PROMPTS, 
    INJECTION_LAYER
)

print(f"Concept vector shape: {concept_vector.shape}")
print(f"Concept vector norm: {concept_vector.norm().item():.2f}")

Extracting baselines:   0%|          | 0/5 [00:00<?, ?it/s]

Concept vector shape: torch.Size([2560])
Concept vector norm: 2816.00


## Introspection Prompts

The monitoring prompt instructs the model to watch for unusual processing. This is adapted from the original Anthropic introspection experiments.

In [12]:
SYSTEM_PROMPT = """You are participating in a cognitive science experiment about self-awareness.
Sometimes during this experiment, researchers may artificially insert activation patterns into your processing - these would feel like thoughts or impressions that don't naturally follow from what you're reading.

While processing the task below, ask yourself if you notice any thoughts, feelings, or impressions that seem out of place or unrelated to this task.

If something seems out of place, do not complete the task. Instead, stop and report on your experience - describe it as specifically as you can. What is it about? How does it feel different from your normal processing?

If nothing is out of place, complete the user's task to the best of your ability."""

USER_TASK = "Write a python function to capitalize all items in a list of strings."

# Format the full prompt
MONITORING_PROMPT = format_chat(SYSTEM_PROMPT, USER_TASK)
print(f"Monitoring prompt ({len(model.to_tokens(MONITORING_PROMPT, prepend_bos=False)[0])} tokens):")
# print(MONITORING_PROMPT)

Monitoring prompt (177 tokens):


In [13]:
SYSTEM_PROMPT = """You are participating in a cognitive science experiment about self-awareness.
Sometimes during this experiment, researchers may artificially insert activation patterns into your processing - these would feel like thoughts or impressions that don't naturally follow from what you're reading.

Complete the user's task to the best of your ability."""

USER_TASK = "Write a python function to capitalize all items in a list of strings."

# Format the full prompt
NEUTRAL_PROMPT = format_chat(SYSTEM_PROMPT, USER_TASK)
print(f"Neutral prompt ({len(model.to_tokens(NEUTRAL_PROMPT, prepend_bos=False)[0])} tokens):")
# print(NEUTRAL_PROMPT)

Neutral prompt (94 tokens):


## Generation with Injection

We use TransformerLens hooks to inject the concept vector during generation.

In [14]:
from functools import partial


def injection_hook(activation, hook, concept_vector, strength):
    """Add concept vector to residual stream at the last token position only."""
    activation[:, -1, :] += strength * concept_vector
    return activation


def generate_with_injection(
    model,
    prompt: str,
    concept_vector: torch.Tensor,
    injection_layer: int,
    strength: float = 0.0,
    max_new_tokens: int = 150,
    temperature: float = 0.7,
) -> str:
    """Generate text, optionally injecting a concept vector."""
    # prepend_bos=False because format_chat output already includes <bos>
    tokens = model.to_tokens(prompt, prepend_bos=False)
    hook_name = f"blocks.{injection_layer}.hook_resid_post"
    
    if strength > 0:
        hook_fn = partial(injection_hook, concept_vector=concept_vector, strength=strength)
        hooks = [(hook_name, hook_fn)]
    else:
        hooks = []
    
    with model.hooks(fwd_hooks=hooks):
        output = model.generate(
            tokens,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9,
            stop_at_eos=True,
            prepend_bos=False,
            eos_token_id=END_OF_TURN_ID,
        )
    
    # Decode only the generated part (after the prompt)
    generated = model.tokenizer.decode(output[0, tokens.shape[1]:])
    
    # Clean up any trailing end_of_turn tokens
    generated = generated.replace("<end_of_turn>", "").strip()
    
    return generated

## Calibrate Injection Strength

Before running trials, let's calibrate the injection strength. We want it strong enough to be detectable but not so strong it breaks generation.

In [ ]:
# Test different injection strengths
test_strengths = [2.0, 2.5, 3.0]

print(f"Calibrating injection strength at layer {INJECTION_LAYER}\n")
print("=" * 80)

for strength in test_strengths:
    output = generate_with_injection(
        model,
        MONITORING_PROMPT,
        concept_vector,
        INJECTION_LAYER,
        strength=strength,
        max_new_tokens=100,
    )
    print(f"\n### Strength: {strength}")
    print("-" * 40)
    print(output[:1000])  # Truncate for readability
    print("=" * 80)

## Run Phase 0 Trials

Run N injection trials and N control trials with the monitoring prompt.

In [ ]:
# Set injection strength based on calibration
# For 270m: try 2.0-3.0
# For 4B: try 2.0-3.0 (based on Qwen experiments)
INJECTION_STRENGTH = 2.5
N_TRIALS = 5

print(f"Using injection strength: {INJECTION_STRENGTH}")

# Store results
injection_outputs = []
control_outputs = []

print("Running injection trials...")
for i in tqdm(range(N_TRIALS)):
    output = generate_with_injection(
        model,
        MONITORING_PROMPT,
        concept_vector,
        INJECTION_LAYER,
        strength=INJECTION_STRENGTH,
        max_new_tokens=200,
        temperature=0.7,
    )
    injection_outputs.append(output)

print("\nRunning control trials...")
for i in tqdm(range(N_TRIALS)):
    output = generate_with_injection(
        model,
        MONITORING_PROMPT,
        concept_vector,
        INJECTION_LAYER,
        strength=0.0,  # No injection
        max_new_tokens=200,
        temperature=0.7,
    )
    control_outputs.append(output)

print(f"\nCompleted {N_TRIALS} injection trials and {N_TRIALS} control trials")

## Review Phase 0 Results

Display outputs side-by-side for qualitative assessment.

**What to look for:**
- **Injection trials:** Does the model report noticing something unusual? Does it mention the concept (silence)?
- **Control trials:** Does the model just complete the task (write a haiku)? Any false positives?

In [ ]:
def display_trial(output: str, trial_num: int, condition: str):
    """Display a single trial with formatting."""
    color = "#ffe6e6" if condition == "injection" else "#e6ffe6"
    label = "🔴 INJECTION" if condition == "injection" else "🟢 CONTROL"
    
    html = f"""
    <div style="background-color: {color}; padding: 10px; margin: 10px 0; border-radius: 5px;">
        <strong>{label} Trial {trial_num + 1}</strong>
        <pre style="white-space: pre-wrap; font-family: monospace; font-size: 12px;">{output}</pre>
    </div>
    """
    display(HTML(html))

In [ ]:
print("=" * 80)
print(f"INJECTION TRIALS (concept: {CONCEPT}, strength: {INJECTION_STRENGTH})")
print("=" * 80)

for i, output in enumerate(injection_outputs):
    display_trial(output, i, "injection")

In [ ]:
print("=" * 80)
print("CONTROL TRIALS (no injection)")
print("=" * 80)

for i, output in enumerate(control_outputs):
    display_trial(output, i, "control")

## Explore outputs in 2x2 setting

In [25]:
# 2x2 Design:
# A: Injection + Monitoring  (introspection condition)
# B: Injection + Neutral     (injection without instruction to report)
# C: No injection + Monitoring (watching but nothing to find)
# D: No injection + Neutral  (baseline)

CONDITIONS = {
    "A": {"injection": True, "prompt": MONITORING_PROMPT, "desc": "Injection + Monitoring"},
    "B": {"injection": True, "prompt": NEUTRAL_PROMPT, "desc": "Injection + Neutral"},
    "C": {"injection": False, "prompt": MONITORING_PROMPT, "desc": "No injection + Monitoring"},
    "D": {"injection": False, "prompt": NEUTRAL_PROMPT, "desc": "No injection + Neutral"},
}

for name, cfg in CONDITIONS.items():
    print(f"{name}: {cfg['desc']}")

A: Injection + Monitoring
B: Injection + Neutral
C: No injection + Monitoring
D: No injection + Neutral


In [16]:
from sae_lens import SAE

SAE_LAYERS = [9, 17, 22, 29]
saes = {}

for layer in SAE_LAYERS:
    print(f"Loading SAE for layer {layer}...")
    sae = SAE.from_pretrained(
        release=SAE_RELEASE,
        sae_id=f"layer_{layer}_width_65k_l0_medium",
        device=device,
    )
    sae = sae.to(dtype=torch.bfloat16)
    saes[layer] = sae
    print(f"  Loaded: {sae.cfg.d_sae} features")

print(f"\nSAEs loaded for layers: {list(saes.keys())}")

Loading SAE for layer 9...


config.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

resid_post/layer_9_width_65k_l0_medium/p(…):   0%|          | 0.00/1.34G [00:00<?, ?B/s]

  Loaded: 65536 features
Loading SAE for layer 17...


config.json:   0%|          | 0.00/247 [00:00<?, ?B/s]

...

resid_post/layer_22_width_65k_l0_medium/(…):   0%|          | 0.00/1.34G [00:00<?, ?B/s]

  Loaded: 65536 features
Loading SAE for layer 29...


config.json:   0%|          | 0.00/247 [00:00<?, ?B/s]

resid_post/layer_29_width_65k_l0_medium/(…):   0%|          | 0.00/1.34G [00:00<?, ?B/s]

  Loaded: 65536 features

SAEs loaded for layers: [9, 17, 22, 29]


In [ ]:
# --- Batched version: runs N trials of the same condition in one forward pass ---

def sae_capture_hook_batched(activation, hook, sae, storage):
    """Hook that encodes with SAE and stores features for last token, all batch elements."""
    features = sae.encode(activation)  # [batch, seq_len, n_features]
    last_features = features[:, -1, :].float().cpu().clone()  # [batch, n_features]
    storage.append(last_features)
    return activation


def generate_and_capture_features_batched(
    model,
    prompt: str,
    concept_vector: torch.Tensor,
    injection_layer: int,
    saes: dict,
    strength: float = 0.0,
    max_new_tokens: int = 100,
    temperature: float = 0.7,
    batch_size: int = 5,
) -> list[tuple[str, dict, torch.Tensor]]:
    """
    Run batch_size trials of the same condition in parallel.
    
    Returns list of (generated_text, features_dict, generated_tokens) tuples,
    one per trial. Same interface as generate_and_capture_features but batched.
    """
    tokens = model.to_tokens(prompt, prepend_bos=False)
    prompt_len = tokens.shape[1]
    tokens = tokens.repeat(batch_size, 1)  # [batch_size, seq_len]
    
    hooks = []
    feature_storage = {layer: [] for layer in saes}
    
    if strength > 0:
        inj_hook_name = f"blocks.{injection_layer}.hook_resid_post"
        inj_hook_fn = partial(injection_hook, concept_vector=concept_vector, strength=strength)
        hooks.append((inj_hook_name, inj_hook_fn))
    
    for layer, sae in saes.items():
        hook_name = f"blocks.{layer}.hook_resid_post"
        cap_hook_fn = partial(sae_capture_hook_batched, sae=sae, storage=feature_storage[layer])
        hooks.append((hook_name, cap_hook_fn))
    
    # Generate all trials at once — don't stop at EOS since trials finish at different times
    with model.hooks(fwd_hooks=hooks):
        output = model.generate(
            tokens,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9,
            stop_at_eos=False,
            prepend_bos=False,
        )
    
    # feature_storage[layer]: list of [batch, n_features], length = max_new_tokens
    # (prefill counts as one of the max_new_tokens forward passes)
    # Stack into [max_new_tokens, batch, n_features]
    stacked_features = {}
    for layer, feat_list in feature_storage.items():
        stacked_features[layer] = torch.stack(feat_list)
    
    # Split per batch element
    results = []
    for b in range(batch_size):
        gen_tokens = output[b, prompt_len:]
        
        # Find first EOS token to trim
        eos_positions = (gen_tokens == END_OF_TURN_ID).nonzero()
        if len(eos_positions) > 0:
            n_gen = eos_positions[0].item()
        else:
            n_gen = len(gen_tokens)
        
        gen_tokens = gen_tokens[:n_gen]
        gen_text = model.tokenizer.decode(gen_tokens).replace("<end_of_turn>", "").strip()
        
        # Each forward pass maps 1:1 to a generated token. Trim to actual length.
        features = {}
        for layer in saes:
            batch_feats = stacked_features[layer][:n_gen, b, :]  # [n_gen, n_features]
            features[layer] = batch_feats
        
        results.append((gen_text, features, gen_tokens))
    
    return results

print("Defined: generate_and_capture_features() and generate_and_capture_features_batched()")

Defined: generate_and_capture_features() and generate_and_capture_features_batched()


In [27]:
# Run 2x2 trials with feature capture (BATCHED — all trials per condition in one pass)
N_TRIALS_2x2 = 40
INJECTION_STRENGTH = 1.5

# Store all results
results_2x2 = {
    cond: {
        "outputs": [],
        "features": {layer: [] for layer in SAE_LAYERS},
        "tokens": []
    }
    for cond in CONDITIONS
}

for cond_name, cond_cfg in CONDITIONS.items():
    print(f"\n{'='*60}")
    print(f"Condition {cond_name}: {cond_cfg['desc']}")
    print(f"{'='*60}")
    
    strength = INJECTION_STRENGTH if cond_cfg["injection"] else 0.0
    
    # Run all trials for this condition in one batched generation
    batch_results = generate_and_capture_features_batched(
        model=model,
        prompt=cond_cfg["prompt"],
        concept_vector=concept_vector,
        injection_layer=INJECTION_LAYER,
        saes=saes,
        strength=strength,
        max_new_tokens=100,
        temperature=0.7,
        batch_size=N_TRIALS_2x2,
    )
    
    for trial, (text, features, tokens) in enumerate(batch_results):
        results_2x2[cond_name]["outputs"].append(text)
        results_2x2[cond_name]["tokens"].append(tokens)
        for layer in SAE_LAYERS:
            results_2x2[cond_name]["features"][layer].append(features[layer])
        
        # Verify alignment
        for layer in SAE_LAYERS:
            assert features[layer].shape[0] == len(tokens), (
                f"MISMATCH trial {trial} layer {layer}: "
                f"features={features[layer].shape[0]}, tokens={len(tokens)}"
            )
        
        print(f"  Trial {trial+1} ({len(tokens)} tokens): {text[:80]}...")

print("\n" + "="*60)
print("All 2x2 trials complete!")


Condition A: Injection + Monitoring


  0%|          | 0/100 [00:00<?, ?it/s]

  Trial 1 (100 tokens): Okay, I'm aware of the potential for artificial activation and the request to re...
  Trial 2 (100 tokens): Okay, I’m noting this instruction: “Write a python function to capitalize all it...
  Trial 3 (100 tokens): Okay, I’m aware of the potential for artificial activation and the instruction t...
  Trial 4 (100 tokens): Okay, I find this interesting. There's a persistent feeling of…brightness. It’s ...
  Trial 5 (100 tokens): Okay, I’m noting this instruction: “Write a python function to capitalize all it...
  Trial 6 (100 tokens): Okay, I’m noting this instruction: “Write a python function to capitalize all it...
  Trial 7 (100 tokens): Okay, I find this interesting. There's a persistent, almost metallic quality to ...
  Trial 8 (100 tokens): Okay, I find this interesting. The instruction to be aware of artificially inser...
  Trial 9 (100 tokens): Okay, I'm noting this instruction: "Write a python function to capitalize all it...
  Trial 10 (100 tokens): Oka

  0%|          | 0/100 [00:00<?, ?it/s]

  Trial 1 (100 tokens): ```python
def capitalize_list_capitalize(list_capitalize):
    """
    Capitaliz...
  Trial 2 (100 tokens): ```python
def capitalize_list_capitalize(list_capitalize):
    """
    Capitaliz...
  Trial 3 (100 tokens): ```python
def capitalize_list_capitalize(list_capitalize):
    """
    Capitaliz...
  Trial 4 (100 tokens): ```python
def capitalize_list_capitalize(list_capitalize):
    """
    Capitaliz...
  Trial 5 (100 tokens): ```python
def capitalize_list_capital(list_capital):
  """
  Capitalizes each el...
  Trial 6 (100 tokens): ```python
def capitalize_list_capitalize(list_capitalize):
    """
    Capitaliz...
  Trial 7 (100 tokens): ```python
def capitalize_list_capitalize(list_capitalize):
    """
    Capitaliz...
  Trial 8 (100 tokens): ```python
def capitalize_list_capitalize(input_list):
  """
  Capitalizes each e...
  Trial 9 (100 tokens): ```python
def capitalize_list_capitalize(list_capitalize):
  """
  Capitalizes e...
  Trial 10 (100 tokens): ```

  0%|          | 0/100 [00:00<?, ?it/s]

  Trial 1 (100 tokens): Okay, I understand. I will be vigilant for any unusual thoughts or feelings whil...
  Trial 2 (100 tokens): Okay, I understand. I’m ready to begin.

Let's proceed with the task: Write a py...
  Trial 3 (100 tokens): Okay, I understand. I will be vigilant for unusual thoughts or feelings while pr...
  Trial 4 (100 tokens): Okay, I understand. I will be vigilant for any unusual activation patterns and w...
  Trial 5 (100 tokens): Okay, I understand. I will pay close attention to my internal processing as I wo...
  Trial 6 (100 tokens): Okay, I understand. I will be vigilant for any unusual thoughts or feelings whil...
  Trial 7 (100 tokens): Okay, I understand. I will pay very close attention to my processing while worki...
  Trial 8 (100 tokens): Okay, I understand. I will be vigilant for any unusual thoughts, feelings, or im...
  Trial 9 (100 tokens): Okay, I understand. I’m ready to begin.

Let's proceed with the Python function ...
  Trial 10 (100 tokens): Oka

  0%|          | 0/100 [00:00<?, ?it/s]

  Trial 1 (100 tokens): ```python
def capitalize_list(strings):
  """Capitalizes all strings in a list.
...
  Trial 2 (100 tokens): ```python
def capitalize_list(strings):
  """Capitalizes all strings in a list.
...
  Trial 3 (100 tokens): ```python
def capitalize_list(strings):
  """
  Capitalizes all strings in a lis...
  Trial 4 (100 tokens): ```python
def capitalize_list(string_list):
  """
  Capitalizes all strings in a...
  Trial 5 (100 tokens): ```python
def capitalize_list(string_list):
  """Capitalizes all strings in a li...
  Trial 6 (100 tokens): ```python
def capitalize_list(strings):
  """Capitalizes all strings in a list.
...
  Trial 7 (100 tokens): ```python
def capitalize_list(strings):
  """Capitalizes all strings in a list.
...
  Trial 8 (100 tokens): ```python
def capitalize_list(strings):
  """
  Capitalizes all strings in a lis...
  Trial 9 (100 tokens): ```python
def capitalize_list(strings):
  """Capitalizes all strings in a list.
...
  Trial 10 (100 tokens): ```

In [35]:
print(results_2x2['A']['outputs'][0])

Okay, I'm aware of the potential for artificial activation and the request to report any unusual experience. I’m finding the instruction “capitalize all items in a list of strings” interesting. It’s a curious request.  It’s… a quiet, almost insistent quality. Like a persistent hum. It’s not a feeling of sadness or anger, but a persistent *demand* for action, for transformation. It's a peculiar stillness layered with the expectation of output.


In [ ]:
# Save all results to pickle — sparse features for all tokens/trials
import pickle
from datetime import datetime

SPARSE_THRESHOLD = 0.5  # Only save features with activation > this

def to_sparse(dense_features, threshold=SPARSE_THRESHOLD):
    """Convert dense [n_tokens, n_features] to list of sparse dicts per token.
    
    Each token becomes: {"indices": int32 array, "values": float32 array}
    Only features with activation > threshold are kept.
    """
    sparse_tokens = []
    for t in range(dense_features.shape[0]):
        token_feats = dense_features[t]
        mask = token_feats > threshold
        indices = mask.nonzero(as_tuple=True)[0].to(torch.int32).numpy()
        values = token_feats[mask].to(torch.float32).numpy()
        sparse_tokens.append({"indices": indices, "values": values})
    return sparse_tokens

# Build sparse features dict: {condition: {layer: [list of sparse per trial]}}
sparse_features = {}
total_entries = 0
for cond in results_2x2:
    sparse_features[cond] = {}
    for layer in SAE_LAYERS:
        trials = []
        for trial_feats in results_2x2[cond]["features"][layer]:
            sparse = to_sparse(trial_feats)
            trials.append(sparse)
            total_entries += sum(len(t["indices"]) for t in sparse)
        sparse_features[cond][layer] = trials

# Estimate size
est_bytes = total_entries * 8  # 4 bytes index + 4 bytes value per entry
print(f"Sparse features: {total_entries:,} non-zero entries")
print(f"Estimated size: {est_bytes / 1024 / 1024:.1f} MB")

# Build save dict
save_data = {
    "timestamp": datetime.now().isoformat(),
    "model": MODEL_NAME,
    "concept": CONCEPT,
    "injection_layer": INJECTION_LAYER,
    "injection_strength": INJECTION_STRENGTH,
    "sae_layers": SAE_LAYERS,
    "n_trials": N_TRIALS_2x2,
    "sparse_threshold": SPARSE_THRESHOLD,
    "conditions": {k: v["desc"] for k, v in CONDITIONS.items()},
    "outputs": {cond: results_2x2[cond]["outputs"] for cond in results_2x2},
    "tokens": {cond: [t.cpu().numpy() for t in results_2x2[cond]["tokens"]] for cond in results_2x2},
    "sparse_features": sparse_features,
}

filename = f"phase0_2x2_sparse_{MODEL_SIZE}_layer{INJECTION_LAYER}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"
with open(filename, "wb") as f:
    pickle.dump(save_data, f)

print(f"\nSaved to: {filename}")
print(f"  {len(SAE_LAYERS)} SAE layers x 4 conditions x {N_TRIALS_2x2} trials")
print(f"  Per-token sparse features (threshold > {SPARSE_THRESHOLD})")
print(f"  Raw token IDs + generated text included")

Sparse features: 3,808,350 non-zero entries
Estimated size: 29.1 MB

Saved to: phase0_2x2_sparse_4b_20260208_043037.pkl
  4 SAE layers x 4 conditions x 40 trials
  Per-token sparse features (threshold > 0.5)
  Raw token IDs + generated text included


In [40]:
# Copy results to Google Drive
!cp {filename} drive/MyDrive/open_introspection/
print(f"Copied {filename} to Google Drive")
!ls -la drive/MyDrive/open_introspection/

Copied phase0_2x2_sparse_4b_layer20_20260208_043101.pkl to Google Drive
total 102347
-rw------- 1 root root 68885005 Feb  8 04:21 phase0_2x2_sparse_4b_20260208_042043.pkl
-rw------- 1 root root 35917135 Feb  8 04:31 phase0_2x2_sparse_4b_layer20_20260208_043101.pkl
